In [1]:
import numpy as np
import scipy.sparse as sp
import sknetwork as sn
import abcd_graph
import matplotlib.pyplot as plt
from tqdm import tqdm

from fstar import *

In [2]:
XIS = np.linspace(0.2, 0.9, 8)
REPS = 3
n = 10000
nout = 0

fine = sn.clustering.Leiden(resolution=10.0, n_aggregations=4)
med = sn.clustering.Leiden(resolution=2.5, n_aggregations=4)
coarse = sn.clustering.Leiden(resolution=1.0, n_aggregations=4)

similarity = np.empty((3, len(XIS), REPS))
ga_similarity = np.empty_like(similarity)
num = np.empty((4, len(XIS), REPS), dtype="int64")

with tqdm(total = len(XIS) * REPS) as pbar:
    for i, xi in enumerate(XIS):
        for j in range(REPS):
            params = abcd_graph.ABCDParams(vcount=n, xi=xi, num_outliers=nout, max_community_size=250)
            graph = abcd_graph.ABCDGraph(params).build()
            labels = np.array(
                [i for i, com in enumerate(graph.communities) for _ in com.vertices]
            )
            labels[labels == np.max(labels)] = -1
            labels = clustering_array_to_sparse(labels)
            adjacency = graph.exporter.to_sparse_adjacency_matrix().astype("int32")
            el = node_clustering_to_edge_clustering(adjacency, labels)
            num[3,i,j] = labels.shape[0]

            f = clustering_array_to_sparse(np.asarray(fine.fit_predict(adjacency)))
            num[0,i,j] = f.shape[0]
            similarity[0,i,j] = fstar(f, labels)
            ga_similarity[0,i,j] = fstar(el, node_clustering_to_edge_clustering(adjacency, f))

            m = clustering_array_to_sparse(np.array(med.fit_predict(adjacency)))
            num[1,i,j] = m.shape[0]
            similarity[1,i,j] = fstar(m, labels)
            ga_similarity[1,i,j] = fstar(el, node_clustering_to_edge_clustering(adjacency, m))

            c = clustering_array_to_sparse(np.asarray(coarse.fit_predict(adjacency)))
            num[2,i,j] = c.shape[0]
            similarity[2,i,j] = fstar(c, labels)
            ga_similarity[2,i,j] = fstar(el, node_clustering_to_edge_clustering(adjacency, c))
            pbar.update()

np.save("similarity.npy", similarity)
np.save("ga_similarity.npy", ga_similarity)
np.save("num.npy", num)

  0%|          | 0/24 [00:00<?, ?it/s]

 46%|████▌     | 11/24 [01:51<02:10, 10.08s/it]